In [1]:
import pandas as pd
import numpy as np

C:\Users\soura\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\soura\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\soura\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# reading users file
u_cols = ['user_id','age','sex','occupation','zip_code']
users = pd.read_csv('data/u.user' , sep='|',names = u_cols)
# reading rating file
r_cols = ['user_id','movie_id','rating','unix_timestamp']
ratings = pd.read_csv('data/u.data',sep ='\t',names = r_cols)
# reading items file:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('data/u.item', sep='|', names=i_cols,
encoding='latin-1')

# After loading the dataset, we should look at the content of each file (users, ratings, items).

# Looking at the user file
print("\nUser Data :")
print("shape : ", users.shape)
print(users.head())

# We have 943 users in the dataset and each user has 5 features, i.e. user_ID, age, sex, occupation and zip_code. Now let’s look at the ratings file.

# Ratings Data
print("\nRatings Data :")
print("shape : ", ratings.shape)
print(ratings.head())

# We have 100k ratings for different user and movie combinations. Now finally examine the items file.

# Item Data
print("\nItem Data :")
print("shape : ", items.shape)
print(items.head())


User Data :
shape :  (943, 5)
   user_id  age sex  occupation zip_code
0        1   24   M  technician    85711
1        2   53   F       other    94043
2        3   23   M      writer    32067
3        4   24   M  technician    43537
4        5   33   F       other    15213

Ratings Data :
shape :  (100000, 4)
   user_id  movie_id  rating  unix_timestamp
0      196       242       3       881250949
1      186       302       3       891717742
2       22       377       1       878887116
3      244        51       2       880606923
4      166       346       1       886397596

Item Data :
shape :  (1682, 24)
   movie id        movie title release date  video release date  \
0         1   Toy Story (1995)  01-Jan-1995                 NaN   
1         2   GoldenEye (1995)  01-Jan-1995                 NaN   
2         3  Four Rooms (1995)  01-Jan-1995                 NaN   
3         4  Get Shorty (1995)  01-Jan-1995                 NaN   
4         5     Copycat (1995)  01-Jan-1995     

In [3]:
r_cols = ['user_id','monie_id','rating','unix_timestamp']
rating_train = pd.read_csv('data/ua.base', sep ='\t',names = r_cols)
rating_test = pd.read_csv('data/ua.test',sep = '\t' , names= r_cols)
print(rating_train.shape , rating_test.shape)

(90570, 4) (9430, 4)


# Building collaborative filtering model from scratch
We will recommend movies based on user-user similarity and item-item similarity. For that, first we nees to calculate the number of unique users and movies.

In [4]:
n_users = ratings.user_id.nunique()
n_items = ratings.movie_id.nunique()

Now, We will create a user-item matrix which can be used to calculate the similarity between users and items.

In [5]:
data_matrix = np.zeros((n_users,n_items))
for line in ratings.itertuples():
    data_matrix[line[1]-1 , line[2]-1] = line[3]
    

Now, we will calculate the similarity. We can use the pairwise_distance function from sklearn to calculate the cosine similarity.

In [6]:
from sklearn.metrics.pairwise import pairwise_distances

In [7]:
user_similarity = pairwise_distances(data_matrix, metric = 'cosine')
item_similarity = pairwise_distances(data_matrix.T,metric = 'cosine')


This gives us the item-item and user-user similarity in an array form. the next step is to make prediction based on these similarities. Let's define a function to do just that.

In [23]:
def predict(ratings, similarity, type ='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis = 1)
        # we use np.newaxis so that mean_user_rating has same format as ratins
        ratings_diff = (ratings - mean_user_rating[:,np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

Finally, we will make predictions based on user similarity and item similarity.

In [25]:
user_prediction = predict(data_matrix, user_similarity, type='user')
item_prediction = predict(data_matrix, item_similarity, type='item')

As it turns out, we also have a library which generates all these recommendations automatically. Let us now learn how to create a recommendation engine using turicreate in Python. To get familiar with turicreate and to install it on your machine, refer here.

In [26]:
import turicreate

ModuleNotFoundError: No module named 'turicreate'